# Nginx

Nginx viene a ser un competidor serio para Apache en asuntos de desempeño y adopción de nuevas tecnologías como lo han sido spdy y http2.

Es utilizado por proveedores de servicio para volúmenes altos de tráfico, como balanceador, como proxy y por supuesto como servidor web para contenido dinámico.

La configuración de Nginx podría resultar más clara para algunos administradores por lo que algunas veces es también utilizado por preferencia profesional.

Adicionalmente tiene capacidades de "correo", que significa que puede hablar protocolos de SMTP, POP3 e IMAP4 en capacidad de Proxy, por lo que puede ser también usado para crear clusters de servidores de correo y servidores web. También se encuentra empotrado en herramientas como Zimbra Collaboration Suite.

In [ ]:
sudo apt-get -y install nginx
echo ; find /etc/nginx -type f

### Contenido dinámico

Cuando instalamos agregados a Apache para contenido dinámico la mayoría usaron alguna forma de conectores de CGI, a diferencia de Php "normal" el cual usa un conector especial para Apache por medio de libapache2-php.

En el caso de Nginx no existen conectores especiales sino que todas las plataformas usan el conector de FastCGI, el cual ha sido optimizado para funcionar correctamente con cualquiera de los lenguajes disponibles.

Para habilitar soporte de Php vamos a utilizar la versión de Php-fpm, pero puede utilizar Php HHVM si así lo desea.

Para activar soporte de Php debe editar el archivo de sitio, en este caso el ``default``:

**Archivo:** ``/etc/nginx/sites-available/default``

**Cambio:** Vamos a activar soporte para Php, incluyendo un archivo de configuración. Solamente debe **descomentar** las líneas necesarias.

```nginx
        location ~ \.php$ {
                include snippets/fastcgi-php.conf;
        #
        #       # With php7.0-cgi alone:
        #       fastcgi_pass 127.0.0.1:9000;
        #       # With php7.0-fpm:
                fastcgi_pass unix:/run/php/php7.0-fpm.sock;
        }
```

En el mismo archivo, debe también activar soporte para ``index.php`` de forma similar a como lo hicimos en Apache.

```nginx
        # Add index.php to the list if you are using PHP
        index index.php index.html index.htm index.nginx-debian.html;
```

Reiniciamos Nginx y mostramos el estado del servicio.


In [ ]:
sudo systemctl restart nginx
sudo systemctl status nginx | cat

Descomprimimos el código de Drupal dentro de /var/www/html que es el directorio default de Apache.

En un sistema en producción se recomienda crear un directorio para cada virtualhost, o un único virtualhost para drupal y hacer uso de la función de "Multi tenant" para trabajar varios clientes con una sola instancia de la aplicación.

Importante que el código queda con derechos de **www-data**, usuario que utiliza Debian para el servicio de web de Apache. Esto porque Drupal debe poder manipular archivos dentro de la carpeta.

In [ ]:
cat << EOF | sudo tee /var/www/html/phpinfo.php
<?php
phpinfo();
?>
EOF

### Prueba de código

Ahora puede visitar:

http://10.42.25.3X/phpinfo.php

Es importante que esta función de phpinfo() de ser utilizada solamente para verificar el funcionamiento de Php integrado con el servidor web, y debe ser removida luego de ser usada. De forma alternativa puede restringir acceso al archivo por Ip, contraseña u otro método sin embargo no es frecuente su uso.

En el contexto de seguridad se considera una vulnerabilidad el que se encuentre presente un archivo con esta función, dado que expone al atacante detalles del servidor que podrían ser utilizados para escalar o planear un ataque.